<a href="https://colab.research.google.com/github/chen0126/pytorch-training/blob/main/hw2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import csv
import numpy as np
import numpy
import json

with open("/hw2/30702/scale.json","r") as f:
    scales = json.load(f)

conv1_weights = np.loadtxt(("/hw2/30702/conv1.weight.csv") , delimiter="," , skiprows = 0)
conv1_weights = conv1_weights.reshape(6,3,5,5)

conv2_weights = np.loadtxt(("/hw2/30702/conv2.weight.csv") , delimiter="," , skiprows = 0)
conv2_weights = conv2_weights.reshape(16,6,5,5)

fc1_weights = np.loadtxt(("/hw2/30702/fc1.weight.csv") , delimiter="," , skiprows = 0)
fc1_weights = fc1_weights.reshape(120,400)

fc2_weights = np.loadtxt(("/hw2/30702/fc2.weight.csv") , delimiter="," , skiprows = 0)
fc2_weights = fc2_weights.reshape(84,120)

fc3_weights = np.loadtxt(("/hw2/30702/fc3.weight.csv") , delimiter="," , skiprows = 0)
fc3_weights = fc3_weights.reshape(10,84)

fc3_bias = np.loadtxt(("/hw2/30702/fc3.bias.csv") , delimiter="," , skiprows = 0)
fc3_bias = fc3_bias.reshape(10)

input_data = np.loadtxt(("/hw2/30702/input.csv") , delimiter="," , skiprows = 0)
input_data = input_data.reshape(1,3,32,32)
print("input_data.shape[1]: ",input_data.shape[1])
print("weights.shape[0]: ",fc1_weights.shape[0])

# with open("/content/scale.json","r") as f:
#     scales = json.load(f)

# conv1_weights = np.loadtxt(("/content/weights/conv1.weight.csv") , delimiter="," , skiprows = 0)
# conv1_weights = conv1_weights.reshape(6,3,5,5)

# conv2_weights = np.loadtxt(("/content/weights/conv2.weight.csv") , delimiter="," , skiprows = 0)
# conv2_weights = conv2_weights.reshape(16,6,5,5)

# fc1_weights = np.loadtxt(("/content/weights/fc1.weight.csv") , delimiter="," , skiprows = 0)
# fc1_weights = fc1_weights.reshape(120,400)

# fc2_weights = np.loadtxt(("/content/weights/fc2.weight.csv") , delimiter="," , skiprows = 0)
# fc2_weights = fc2_weights.reshape(84,120)

# fc3_weights = np.loadtxt(("/content/weights/fc3.weight.csv") , delimiter="," , skiprows = 0)
# fc3_weights = fc3_weights.reshape(10,84)

# fc3_bias = np.loadtxt(("/content/weights/fc3.bias.csv") , delimiter="," , skiprows = 0)
# fc3_bias = fc3_bias.reshape(10)

# input_data = np.loadtxt(("/content/activations/input.csv") , delimiter="," , skiprows = 0)
# input_data = input_data.reshape(1,3,32,32)

# #output_data = np.loadtxt(("/hw2/1300/output.csv") , delimiter="," , skiprows = 0)
# #output_data = output_data.reshape(10)


FileNotFoundError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# New Section

In [ ]:
import numba as nb

@nb.jit()
def convolution(N,M,P,Q,R,S,C,input_data,weights):
    output_data = np.zeros((N,M,P,Q))
    for n in range(N): 
      for m in range(M):
        for p in range(P):
          for q in range(Q):
            output_data[n][m][p][q] = 0
            for r in range(R):
              for s in range(S):
                for c in range(C):
                  h = p + r
                  w = q + s
                  output_data[n][m][p][q] += input_data[n][c][h][w] * weights[m][c][r][s]
    return output_data

@nb.jit()
def pooling(N,M,P,Q,input_data):
    output_data = np.zeros((N,M,P,Q))
    for n in range(N): 
      for m in range(M):
        for p in range(P):
          for q in range(Q):
            output_data[n][m][p][q] = max(input_data[n][m][2*p][2*q],input_data[n][m][2*p+1][2*q],input_data[n][m][2*p][2*q+1],input_data[n][m][2*p+1][2*q+1])
    return output_data

@nb.jit()
def fully_connected(N,input_data,weights):
    X = input_data.shape[1]
    Y = weights.shape[0]
    output_data = np.zeros((N,Y))
    for n in range(N):
      for y in range(Y):
        for x in range(X):
          output_data[n][y] += input_data[n][x] * weights[y][x] 
    return output_data

@nb.jit()
def relu(OA):
    OA = np.maximum(OA,0)
    return OA

@nb.jit()
def forward_C(N,x):
    # plt.figure(figsize=(20,10))

    x = x * scales['input_scale']
    x = np.clip(x.round(),-128,127)
    x = convolution(N,6,28,28,5,5,3,x,conv1_weights)
    x = np.clip(x,-2**17,2**17-1)

    # y = x.flatten()
    # plt.subplot(2,3,1)
    # plt.hist(y)
    # plt.title("conv1_partial_sum")
    x = relu(x)

    x = pooling(N,6,14,14,x)
    x = x * scales['conv1_output_scale']
    x = np.clip(x.round(),-128,127)
    x = convolution(N,16,10,10,5,5,6,x,conv2_weights)
    x = np.clip(x,-2**16,2**16-1)

    # y = x.reshape(-1)
    # plt.subplot(2,3,2)
    # plt.hist(y)
    # plt.title("conv2_partial_sum")
    x = relu(x)
    x = pooling(N,16,5,5,x)
    x = x * scales['conv2_output_scale']
    x = np.clip(x.round(),-128,127)

    x = x.reshape(N,-1)

    x = fully_connected(N,x,fc1_weights)
    x = np.clip(x,-2**15,2**15-1)
    # y = x.reshape(-1)
    # plt.subplot(2,3,3)
    # plt.hist(y)
    # plt.title("fc1_partial_sum")
    x = relu(x)
    x = x * scales['fc1_output_scale']
    x = np.clip(x.round(),-128,127)  
    x = fully_connected(N,x,fc2_weights)
    x = np.clip(x,-2**15,2**15-1)
    # y = x.reshape(-1)
    # plt.subplot(2,3,4)
    # plt.hist(y)
    # plt.title("fc2_partial_sum")
    x = relu(x)
    x = x * scales['fc2_output_scale']
    x = np.clip(x.round(),-128,127)
    x = fully_connected(N,x,fc3_weights)
    x = np.clip(x,-2**16,2**16-1)
    # y = x.reshape(-1)
    # plt.subplot(2,3,5)
    # plt.hist(y)
    # plt.title("fc3_partial_sum")
    x = x + fc3_bias 
    x = x * scales['fc3_output_scale']
    x = np.clip(x.round(),-128,127)  

    # plt.show()
    return x

# def test(dataloader):
#     correct = 0
#     total = 0
#     n_inferences = 0

#     with torch.no_grad():
#         for data in dataloader:
#             images, labels = data

#             images = images.numpy()
#             labels = labels.numpy()
            
#             outputs = forward_C(images.shape[0],images)

#             predicted = np.argmax(outputs, 1)
#             total += labels.shape[0]

#             correct += (predicted == labels).sum()

#     return 100 * correct / total

In [ ]:
# score = test(testloader)
# print('Accuracy of the network on the test images: {}%'.format(score))

In [ ]:
import csv
import numpy as np
import numpy
import json
#x=np.arange(12).reshape( (1,3,2,2) )
#weight=np.arange(90).reshape( (5,18) )
# x=np.ones(8)
# x=x.reshape( (1,2,2,2) )
# x=x+7;
# print(x)
input=np.arange(18).reshape( (1,2,3,3) )
weight_conv=np.arange(16).reshape( (2,2,2,2) )
#weight=np.ones(16)
#weight=weight.reshape((2,8))
weight=np.arange(16).reshape( (2,8) )
# x = x.reshape(1,-1)
#print(x.shape[1])
def convolution(N,M,P,Q,R,S,C,input_data,weights):
    output_data = np.zeros((N,M,P,Q))
    for n in range(N): 
      for m in range(M):
        for p in range(P):
          for q in range(Q):
            output_data[n][m][p][q] = 0
            for r in range(R):
              for s in range(S):
                for c in range(C):
                  h = p + r
                  w = q + s
                  output_data[n][m][p][q] += input_data[n][c][h][w] * weights[m][c][r][s]
    return output_data
def fully_connected(N,input_data,weights):
    X = input_data.shape[1]
    Y = weights.shape[0]
    output_data = np.zeros((N,Y))
    for n in range(N):
      for y in range(Y):
        for x in range(X):
          output_data[n][y] += input_data[n][x] * weights[y][x] 
    return output_data
x=convolution(1,2,2,2,2,2,2,input,weight_conv)
x = x.reshape(1,-1)
print(x)
fully_connected(1,x,weight)

[[ 268.  296.  352.  380.  684.  776.  960. 1052.]]


array([[21880., 60024.]])

In [ ]:
import csv

def Prototxt_parse():
  path = "../example/app/build/test.csv"
  fp = open(path, "r")
  data = fp.readlines()
  fp.close()
  for d in data[1:]:
      row = d.split(",")
      # ['N', 'C', 'H', 'W', 'M', 'R', 'S', 'U', 'pad', 'NUM_WORKER']
      global N,C,H,W,M,R,S,U,pad,NUM_WORKER
      N=int(row[0])
      C=int(row[1])
      H=int(row[2])
      W=int(row[3])
      M=int(row[4])
      R=int(row[5])
      S=int(row[6])
      U=int(row[7])
      pad=int(row[8])
      NUM_WORKER=int(row[9])

Prototxt_parse()


E=(H-R+2*pad)/U+1
F=(W-S+2*pad)/U+1
n = (N*M*E*F) / NUM_WORKER
mul_times=C*R*S

for i in range(1,NUM_WORKER+1):
  globals()['duplicated'+str(i)] = []

for i in range(1,NUM_WORKER+1):
  mul_counter=0
  line_counter=0
  outputfile= "pe_"+str(i)+".trace"
  path="../example/app/build/traces/"
  f = open(path+outputfile, "a+")
  for line in f:
    line_counter=line_counter+1
    if 'ST ' in line:
      mul_counter = mul_counter+1
    if mul_counter >= mul_times:
      if 'BARWAIT' in line:
        break
      globals()['duplicated'+str(i)].append(line)
  del(globals()['duplicated'+str(i)][0])
  if i == NUM_WORKER:
    n = n+(N*M*E*F - NUM_WORKER * n)
  for output_times in range(1,n-1):
    for word in globals()['duplicated'+str(i)]:
      if '0x3' in word:

        print(d)
        print(type(d))
  # for d in globals()['duplicated'+str(i)]:
  #   f.write(d)


    #print(line)
    # if line == text:
    #   mul_counter = mul_counter+1
    #   if mul_counter==mul_times:
    #     duplicated.append(line)
  f.close()

print(duplicated1)

for i in NUM_WORKER:
  # start_idx = i * n
  # int end_idx = (i + 1) * n - 1

  # if i == NUM_WORKER - 1:
  #   end_idx = N*M*E*F - 1

In [ ]:
import csv
import pandas as pd
import numpy as np

# for i in range(1,NUM_WORKER+1):
#   mul_counter=0
#   line_counter=0
#   outputfile= "pe_"+str(i)+".trace"
#   path="../example/app/build/traces/"
#   f = open(path+outputfile, "a+")
# globals()['duplicated'+str(i)]
def Prototxt_parse():
  path = "../example/app/build/test.csv"
  fp = open(path, "r")
  data = fp.readlines()
  fp.close()
  for d in data[1:]:
      row = d.split(",")
      # ['N', 'C', 'H', 'W', 'M', 'R', 'S', 'U', 'pad', 'NUM_WORKER']
      global N,C,H,W,M,R,S,U,pad,NUM_WORKER
      N=int(row[0])
      C=int(row[1])
      H=int(row[2])
      W=int(row[3])
      M=int(row[4])
      R=int(row[5])
      S=int(row[6])
      U=int(row[7])
      pad=int(row[8])
      NUM_WORKER=int(row[9])

Prototxt_parse()


E=(H-R+2*pad)/U+1
F=(W-S+2*pad)/U+1
n = (N*M*E*F) / NUM_WORKER
mul_times=C*R*S

for i in range(1,NUM_WORKER+1):
  outputfile= "pe_"+str(i)+".trace"
  path="../example/app/build/traces/"
  globals()['df'+str(i)] =  pd.read_csv(path+outputfile)

print(df1)


In [ ]:
import csv

NUM_WORKER=16

ld_counter=0
store_counter=0
stall_counter=0
in_loop = False
f = open("../example/app/build/traces/pe_1.trace", "a+")
for line in f:
  if 'STARTLOOP ' in line:
    in_loop = True
  if in_loop == 1 :
    if 'LD ' in line:
      ld_counter = ld_counter+1
    if 'STALL ' in line:
      stall_counter = stall_counter+1
    if 'ST ' in line:
      store_counter = store_counter+1

print("ld_counter",ld_counter)
print("store_counter",store_counter)
print("stall_counter",stall_counter)
  # for d in globals()['duplicated'+str(i)]:
  #   f.write(d)


    #print(line)
    # if line == text:
    #   mul_counter = mul_counter+1
    #   if mul_counter==mul_times:
    #     duplicated.append(line)
f.close()